**Задача:**

Разбить данные на тренировочную (70%) и тестовую часть (30%).
Используя pytorch создать нейронки с одним и двумя слоями.
(функции активации Relu и Sigmoid. Tо есть 4 нейронки, однослойные с Relu и Sigmoid и двуслойные 1) Relu 2) Sigmoid и наоборот 1) Sigmoid 2) Relu.)
(обучать тремя способами Adagrad, Adam и SGD)
(В качестве Loss можно использовать MSELoss и L1Loss)

Размер слоёв подбирать самостоятельно.
С помощью tensorboard, контролировать обучения. На графике loss для тестовых данных должен пойти вверх.

In [ ]:



#импортируем необходимые библиотеки
# Импорт необходимых библиотек для работы с тензорами, нейронными сетями и данными
import torch  # Основная библиотека для работы с тензорами и построения моделей нейронных сетей
import torch.nn as nn  # Подмодуль для создания и управления слоями нейронных сетей
from torch.utils.data import DataLoader, TensorDataset, random_split  # Утилиты для работы с датасетами
from torch.utils.tensorboard import SummaryWriter  # Средство логирования для анализа результатов в TensorBoard
import numpy as np                # импорт numpy
import pandas as pd               # импорт pandas
from google.colab import files    # нужно для импорта своего csv файла
from sklearn.model_selection import train_test_split # разделение датасетов
from sklearn.utils import Bunch # импорт для создания экземпляров Bunch, которые используются для хранения данных датасета
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()
ar_data = pd.read_csv('test.csv') # Чтение csv файла

Saving Урюпкина Екатерина.csv to Урюпкина Екатерина (1).csv


In [ ]:
ar_data
ar_data = ar_data.delit

,Unnamed: 0,X_0,X_1,X_2,X_3,X_4,X_5,Y
0,0,0,11.707385,6.966882,0.326175,3.309122,0.314540,1148.087566
1,1,0,8.301674,4.910188,0.503220,6.869669,0.486804,1075.102900
2,2,0,8.688930,4.112900,1.353071,11.211440,0.941891,1074.871180
3,3,1,5.876864,3.488284,1.142987,6.626772,0.792522,1060.564176
4,4,1,14.628475,3.241820,0.928390,2.049884,2.975232,1225.022259
...,...,...,...,...,...,...,...,...
995,995,0,16.418500,3.150983,0.061149,4.383981,1.647581,1268.754462
996,996,0,9.393625,4.505967,0.980309,4.140460,0.676731,1082.533338
997,997,1,1.207236,4.067948,0.014814,2.878209,2.738831,1004.449383
998,998,1,16.300191,5.047399,6.384657,5.043851,1.767543,1336.405907


In [ ]:

# Разделение данных на входные признаки (X) и целевую переменную (y)
input_features = ar_data.iloc[:, :-1].values  # Все столбцы, кроме последнего, используются как входные данные
target_variable = ar_data.iloc[:, -1].values  # Последний столбец служит целевой переменной

# Преобразование данных в тензоры PyTorch
input_tensor = torch.tensor(input_features, dtype=torch.float32)  # Конвертируем X в тензор типа float32
target_tensor = torch.tensor(target_variable, dtype=torch.float32).view(-1, 1)  # Конвертируем y в тензор, изменив его форму

# Нормализация входных данных (X)
input_mean = input_tensor.mean(dim=0)  # Вычисляем среднее значение для каждого признака
input_std = input_tensor.std(dim=0)  # Вычисляем стандартное отклонение для каждого признака
input_tensor = (input_tensor - input_mean) / input_std  # Нормализуем данные


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adagrad, Adam, SGD
from tensorflow.keras.callbacks import TensorBoard


# Разделение данных на входные (X) и целевые (Y)
X = ar_data.iloc[:, 1:-1].values
Y = ar_data.iloc[:, -1].values

# Нормализация данных
scaler_X = StandardScaler()
scaler_Y = StandardScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y.reshape(-1, 1)).flatten()

# Разделение на тренировочные и тестовые данные
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Класс для создания моделей
class NeuralNetwork(Model):
    def __init__(self, layers, activations):
        super(NeuralNetwork, self).__init__()
        self.hidden_layers = []
        for layer_size, activation in zip(layers, activations):
            self.hidden_layers.append(Dense(layer_size, activation=activation))
        self.output_layer = Dense(1)  # Выходной слой для регрессии

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)

# Функция для создания оптимизаторов
def get_optimizer(optimizer_name):
    if optimizer_name == "Adagrad":
        return Adagrad(learning_rate=0.01)
    elif optimizer_name == "Adam":
        return Adam(learning_rate=0.01)
    elif optimizer_name == "SGD":
        return SGD(learning_rate=0.01)
    else:
        raise ValueError("Неизвестный оптимизатор")

# Конфигурации моделей
model_configs = [
    {"layers": [32], "activations": ["relu"], "name": "Relu_SingleLayer"},
    {"layers": [32], "activations": ["sigmoid"], "name": "Sigmoid_SingleLayer"},
    {"layers": [32, 16], "activations": ["relu", "sigmoid"], "name": "Relu_Sigmoid"},
    {"layers": [32, 16], "activations": ["sigmoid", "relu"], "name": "Sigmoid_Relu"}
]

# Обучение моделей
epochs = 300  # Уменьшено количество эпох для ускорения
batch_size = 16

for config in model_configs:
    for opt_name in ["Adagrad", "Adam", "SGD"]:
        # Создание модели
        model = NeuralNetwork(config["layers"], config["activations"])
        optimizer = get_optimizer(opt_name)
        model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

        # Настройка TensorBoard
        log_dir = f"runs/{config['name']}_{opt_name}"
        tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

        # Обучение модели
        history = model.fit(
            X_train, Y_train,
            validation_data=(X_test, Y_test),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1,  # Установка вывода прогресса обучения
            callbacks=[tensorboard_callback]
        )

# Для запуска TensorBoard:
# 1. Убедитесь, что код отработал без ошибок.
# 2. Выполните:
%reload_ext tensorboard
%tensorboard --logdir=runs


Streaming output truncated to the last 5000 lines.
Epoch 129/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0166 - mae: 0.1019 - val_loss: 0.0186 - val_mae: 0.1116
Epoch 130/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0155 - mae: 0.0967 - val_loss: 0.0190 - val_mae: 0.1117
Epoch 131/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0164 - mae: 0.1027 - val_loss: 0.0186 - val_mae: 0.1109
Epoch 132/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0160 - mae: 0.1002 - val_loss: 0.0190 - val_mae: 0.1115
Epoch 133/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0162 - mae: 0.1009 - val_loss: 0.0187 - val_mae: 0.1115
Epoch 134/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0163 - mae: 0.1008 - val_loss: 0.0188 - val_mae: 0.1117
Epoch 135/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0172 - mae: 0.1034 - val_loss: 0.0189 - val_mae: 0.1117
Epoch 136/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0176 - mae: 0.1056 - val_loss: 0.0189 - val_mae: 0.11

KeyboardInterrupt: 

In [ ]:

%reload_ext tensorboard
%tensorboard --logdir=runs
